In [4]:
import itertools
import pandas as pd
import yfinance as yf
import numpy as np

In [5]:
class Company:
    def __init__(self, name, multiplier):
        self.name = name
        self.multiplier = multiplier

In [ ]:
class Portfolio:
    def __init__(self, companies):
        self.companies = companies

    def fetch_data(self, start_date, end_date):
        data_dict = {}

        for company in self.companies:
            data_ADR = yf.download(company.name, start=start_date, end=end_date)
            data_local = yf.download(f'{company.name}.BA', start=start_date, end=end_date)

            if data_ADR.empty or data_local.empty:
                print(f"Data for {company.name} or {company.name}.BA is empty. Skipping this ticker.")
                continue

            data_ADR = data_ADR.rename(columns={'Close': f'Close_{company.name}'})
            data_local = data_local.rename(columns={'Close': f'Close_{company.name}.BA'})
            data = pd.concat([data_ADR[f'Close_{company.name}'], data_local[f'Close_{company.name}.BA']], axis=1)

            data[f'Ratio_{company.name}'] = (data[f'Close_{company.name}.BA'] / data[f'Close_{company.name}']) * company.multiplier
            data_dict[company.name] = data

        return data_dict

    def calculate_composite_index(self, data_dict):
        composite_index = pd.DataFrame(index=data_dict[list(data_dict.keys())[0]].index)
        composite_index['composite_ratio'] = np.nan

        for company in data_dict:
            composite_index['composite_ratio'] += data_dict[company][f'Ratio_{company.name}']

        composite_index['composite_ratio'] /= len(data_dict)

        return composite_index

    def calculate_profit(self, data_dict, composite_index, fx_rate):
        usd_1 = 0
        ars_1 = 0

        for company in self.companies:
            if company.name in data_dict:
                adr_high = data_dict[company.name].iloc[-1][f'Close_{company.name}']
                stock_high = data_dict[company.name].iloc[-1][f'Close_{company.name}.BA']
                composite_high = composite_index.iloc[-1]['composite_ratio']
                ars_1 += stock_high / composite_high
                usd_1 += adr_high / composite_high

        usd_balance = usd_1 - ars_1 * fx_rate

        return usd_balance


def generate_power_set(input_set):
    power_set = []
    for r in range(1, len(input_set) + 1):
        power_set.extend(itertools.combinations(input_set, r))
    return power_set

def main():
    # Define list of companies and their multipliers
    company_data = [
        Company('CEPU', 10),
        Company('CRESY', 1),
        Company('EDN', 20),
        Company('GGAL', 10),
        Company('SUPV', 5),
        Company('IRS', 10),
        Company('LOMA', 5),
        Company('PAM', 25),
        Company('TEO', 5),
        Company('TS', 1),
        Company('TX', 1),
        Company('TGS', 1),
    ]

    # Define start and end dates
    one_month_ago = (pd.Timestamp.today() - pd.DateOffset(months=1)).strftime('%Y-%m
